<h1>Imports</h1>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import MetaTrader5 as mt5
import datetime
mt5.initialize()
brick_6r = pd.read_excel('brick_6r.xlsx')
brick_6r = brick_6r[::-1]
brick_6r = brick_6r.reset_index()
del brick_6r['index']
data = pd.DataFrame(mt5.copy_rates_from_pos('WIN$N', mt5.TIMEFRAME_D1, 0, 200))
data['Data'] = pd.to_datetime(data['time'],unit='s')
condicao_brick = brick_6r['Abertura'] <=  brick_6r['Fechamento']
brick_6r.loc[condicao_brick,'brick'] = 1
brick_6r.loc[~condicao_brick,'brick'] = 0

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   time         200 non-null    int64         
 1   open         200 non-null    float64       
 2   high         200 non-null    float64       
 3   low          200 non-null    float64       
 4   close        200 non-null    float64       
 5   tick_volume  200 non-null    uint64        
 6   spread       200 non-null    int32         
 7   real_volume  200 non-null    uint64        
 8   Data         200 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(4), int32(1), int64(1), uint64(2)
memory usage: 13.4 KB


In [2]:
data['positivo'] = data.close - data.open
data['positivo'] = data['positivo'].apply(lambda x : 1 if x>0 else 0) 

In [3]:
brick_6r['Data'] = pd.to_datetime(brick_6r['Data'])
brick_6r['Data'] = brick_6r['Data'].apply(lambda x: x.date())
brick_6r['Data'] = pd.to_datetime(brick_6r['Data'])
brick_6r.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20001 entries, 0 to 20000
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Data        20001 non-null  datetime64[ns]
 1   Abertura    20001 non-null  float64       
 2   Máxima      20001 non-null  float64       
 3   Mínima      20001 non-null  float64       
 4   Fechamento  20001 non-null  float64       
 5   VWAP D      20001 non-null  float64       
 6   VWAP S      20001 non-null  float64       
 7   VWAP D.1    20001 non-null  float64       
 8   brick       20001 non-null  float64       
dtypes: datetime64[ns](1), float64(8)
memory usage: 1.4 MB


In [4]:
mistura = pd.merge(brick_6r,data,how='inner',on='Data')

In [5]:
mistura

,Data,Abertura,Máxima,Mínima,Fechamento,VWAP D,VWAP S,VWAP D.1,brick,time,open,high,low,close,tick_volume,spread,real_volume,positivo
0,2023-03-02,5105.0,5108.0,5102.5,5102.5,5105.88,5061.02,5105.88,0.0,1677715200,106110.0,106260.0,104430.0,104500.0,5245902,5,17072464,0
1,2023-03-02,5102.5,5104.5,5100.0,5100.0,5105.88,5061.02,5105.88,0.0,1677715200,106110.0,106260.0,104430.0,104500.0,5245902,5,17072464,0
2,2023-03-02,5102.5,5105.0,5099.0,5105.0,5105.88,5061.02,5105.88,1.0,1677715200,106110.0,106260.0,104430.0,104500.0,5245902,5,17072464,0
3,2023-03-02,5102.5,5107.5,5100.0,5100.0,5105.88,5061.02,5105.88,0.0,1677715200,106110.0,106260.0,104430.0,104500.0,5245902,5,17072464,0
4,2023-03-02,5102.5,5105.0,5098.0,5105.0,5105.88,5061.02,5105.88,1.0,1677715200,106110.0,106260.0,104430.0,104500.0,5245902,5,17072464,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14497,2023-11-10,5065.0,5066.5,5062.5,5062.5,5065.39,5106.95,5065.39,0.0,1699574400,120545.0,122010.0,120310.0,121850.0,4540285,5,16495042,1
14498,2023-11-10,5065.0,5067.5,5060.0,5067.5,5065.36,5106.66,5065.36,1.0,1699574400,120545.0,122010.0,120310.0,121850.0,4540285,5,16495042,1
14499,2023-11-10,5067.5,5070.0,5067.0,5070.0,5065.37,5106.60,5065.37,1.0,1699574400,120545.0,122010.0,120310.0,121850.0,4540285,5,16495042,1
14500,2023-11-10,5067.5,5071.0,5065.0,5065.0,5065.40,5106.51,5065.40,0.0,1699574400,120545.0,122010.0,120310.0,121850.0,4540285,5,16495042,1


<h1>Probabidade de ser igual ao anterior 6R</h1>

In [ ]:
igual = 0
for i in range(20000):
    d1 = brick_6r['Dia'][i]
    d2 = brick_6r['Dia'][i+1]
    if d1 == d2:      
        igual += 1 

print(igual/20001)

0.6741662916854158


<h1>Probabidade de virar um brick e o próximo ser igual ao anterior 6R</h1>

In [ ]:
igual = 0
total = 0
_3_bricks = 0
for i in range(19998):
    d1 = brick_6r['Dia'][i]
    d2 = brick_6r['Dia'][i+1]
    d3 = brick_6r['Dia'][i+2]
    d4 = brick_6r['Dia'][i+3]
    if d1 != d2:        
        total += 1
        if d2 == d3:
            igual+= 1 
            if d3 == d4:
                _3_bricks += 1
print(igual/total,_3_bricks/total)

0.671373752877974 0.4481964696853415


<h1>Probabidade ao pivotiar para cima, virar um brik e romper o topo no 6r</h1>

In [ ]:
topos_fundos = pd.read_excel('topos_fundos_6r.xlsx')
topos_fundos = topos_fundos[::-1]
topos_fundos = topos_fundos.reset_index()
del topos_fundos['index']
topos_fundos
brick_6r = brick_6r.reset_index(drop=True)
mistura = brick_6r.merge(topos_fundos,on='Data',how='inner')
mistura = mistura.drop_duplicates(subset='Data')
mistura = mistura.reset_index()
mistura['Data'] = mistura['Data'].str.replace(' ','-')
mistura['Data'] = mistura['Data'].str.replace('/','-')
mistura['Data'] = mistura['Data'].str.split('-')
datas = []
for dia,mes,ano,_ in mistura['Data'].values:
    datas.append(f'{ano}-{mes}-{dia}')
mistura['Data'] = datas
mistura


,index,Data,Abertura,Máxima,Mínima,Fechamento,VWAP D,VWAP S,VWAP D.1,Dia,Topos e Fundos
0,0,2023-02-03,5105.0,5108.0,5102.5,5102.5,5105.88,5061.02,5105.88,0.0,5108.0
1,1,2023-02-03,5102.5,5105.0,5099.0,5105.0,5105.88,5061.02,5105.88,1.0,5099.0
2,2,2023-02-03,5102.5,5107.5,5100.0,5100.0,5105.88,5061.02,5105.88,0.0,5107.5
3,3,2023-02-03,5102.5,5105.0,5098.0,5105.0,5105.88,5061.02,5105.88,1.0,5098.0
4,4,2023-02-03,5105.0,5107.5,5105.0,5107.5,5105.88,5061.02,5105.88,1.0,5108.0
...,...,...,...,...,...,...,...,...,...,...,...
6466,7183,2023-10-11,5075.0,5078.0,5072.5,5072.5,5065.05,5108.45,5065.05,0.0,5078.0
6467,7184,2023-10-11,5067.5,5070.0,5063.5,5070.0,5065.15,5107.98,5065.15,1.0,5063.5
6468,7185,2023-10-11,5070.0,5074.0,5067.5,5067.5,5065.39,5107.45,5065.39,0.0,5074.0
6469,7186,2023-10-11,5065.0,5067.5,5060.0,5067.5,5065.36,5106.66,5065.36,1.0,5060.0


In [ ]:
mt5.initialize()
ativo ='WDO$N'
regiao = {}
dia = datetime.datetime(year=2023,month=1,day=1)
dia_1 = dia + datetime.timedelta(days=1)
for i in range(400):
    ticks = pd.DataFrame(mt5.copy_ticks_range(ativo,dia,dia_1,mt5.COPY_TICKS_ALL))
    grupo = ticks.groupby('last').agg({'volume':'sum'}) 
    grupo['media_volume'] = grupo['volume'].rolling(window = 8,min_periods=1).mean()
    abaixo_media = grupo.loc[grupo['volume']<grupo['media_volume']]
    if len(grupo['volume'])!=0:
            regiao[f'{dia.date()}'] = abaixo_media

            
    if dia == datetime.datetime(year=2023,month=10,day=13):
        break
        
    dia = dia + datetime.timedelta(days=1)
    dia_1 = dia_1 + datetime.timedelta(days=1)



In [ ]:
regiao['2023-10-09']

,volume,media_volume
last,,
5141.5,3290,3538.125
5147.5,35039,35158.500
5148.5,38807,39205.500
5149.0,31076,38978.000
5149.5,25936,37945.750
...,...,...
5196.5,953,1404.500
5197.0,1020,1059.000
5198.5,888,982.750


In [29]:
reversao = 0
gain = 0
for i in range(len(mistura['Data'])-6):
    if (mistura['Topos e Fundos'][i+2] > mistura['Topos e Fundos'][i]) and (mistura['Topos e Fundos'][i+1] < mistura['Topos e Fundos'][i-1]) and (mistura['Topos e Fundos'][i+3] > mistura['Topos e Fundos'][i+1]) and (mistura['Topos e Fundos'][i+3] < mistura['Topos e Fundos'][i+2])\
    and mistura['Topos e Fundos'][i+3] in regiao[mistura['Data'][i+3][:10]].index:
        reversao +=1
        if mistura['Topos e Fundos'][i+4] > mistura['Topos e Fundos'][i+2]:
            gain+= 1
print(gain/reversao)

0.5179856115107914


<h1>Estudo ENOR virada de brik com agressão contrária ao movimento</h1>

In [53]:
brick_4r_cheio = pd.read_excel('brick4r_cheio.xlsx')
brick_4r_pro = pd.read_excel('brick4r_pro.xlsx')
topos_fundos_4r_cheio = pd.read_excel('topos_fundos_4t-cheio.xlsx')
topos_fundos_4r_pro = pd.read_excel('topos_fundos_4t-pro.xlsx')
agressao_4r_cheio = pd.read_excel('agresao_4r_cheio.xlsx')
agressao_4r_pro = pd.read_excel('agrassao_4r_pro.xlsx')
topos_fundos_agressao_4r_cheio = brick_4r_cheio.merge(agressao_4r_cheio,on='Data')
topos_fundos_agressao_4r_cheio = topos_fundos_agressao_4r_cheio[::-1]
topos_fundos_agressao_4r_cheio.reset_index(drop=True,inplace=True)
topos_fundos_agressao_4r_cheio

,Data,Abertura,Máxima,Mínima,Fechamento,VWAP D,Prior Cote,VWAP S,VWAP D.1,Quantidade de Agressão - 4R-cheio
0,01/06/2023 09:30:23.841,5062.5,5065.0,5061.0,5061.0,5068.45,5102.65,5074.65,5068.45,-295
1,01/06/2023 09:30:40.552,5061.0,5061.5,5059.5,5059.5,5068.35,5102.65,5074.64,5068.35,-250
2,01/06/2023 09:30:48.500,5059.5,5060.0,5058.0,5058.0,5068.24,5102.65,5074.63,5068.24,-40
3,01/06/2023 09:30:54.717,5058.0,5058.0,5056.5,5056.5,5068.22,5102.65,5074.63,5068.22,-45
4,01/06/2023 09:30:54.751,5058.0,5059.5,5056.0,5059.5,5067.86,5102.65,5074.61,5067.86,85
...,...,...,...,...,...,...,...,...,...,...
26838,13/10/2023 17:59:28.668,5094.0,5095.5,5094.0,5095.5,5085.11,5072.26,5097.94,5085.11,35
26839,13/10/2023 17:59:59.091,5095.5,5097.0,5095.5,5097.0,5085.11,5072.26,5097.94,5085.11,60
26840,13/10/2023 17:59:59.091,5095.5,5097.0,5095.5,5097.0,5085.11,5072.26,5097.94,5085.11,40
26841,13/10/2023 17:59:59.091,5097.0,5098.0,5097.0,5097.0,5085.11,5072.26,5097.94,5085.11,60


In [61]:
topos_fundos_agressao_4r_cheio_ = topos_fundos_agressao_4r_cheio.merge(topos_fundos_4r_cheio,on='Data')
topos_fundos_agressao_4r_cheio_ = topos_fundos_agressao_4r_cheio_.drop_duplicates('Data')
topos_fundos_agressao_4r_cheio_.reset_index(drop=True,inplace=True)
topos_fundos_agressao_4r_cheio_

,Data,Abertura,Máxima,Mínima,Fechamento,VWAP D,Prior Cote,VWAP S,VWAP D.1,Quantidade de Agressão - 4R-cheio,Topos/fundos 4r_cheio
0,01/06/2023 09:30:54.751,5058.0,5059.5,5056.0,5059.5,5067.86,5102.65,5074.61,5067.86,85,5056.0
1,01/06/2023 09:35:48.297,5065.5,5067.5,5064.0,5064.0,5067.44,5102.65,5074.57,5067.44,-285,5067.5
2,01/06/2023 09:45:21.548,5052.0,5053.5,5049.0,5053.5,5065.66,5102.65,5074.43,5065.66,5,5049.0
3,01/06/2023 09:54:24.608,5071.5,5073.5,5070.0,5070.0,5064.16,5102.65,5074.19,5064.16,120,5073.5
4,01/06/2023 09:56:12.835,5068.5,5070.0,5066.5,5070.0,5064.34,5102.65,5074.18,5064.34,220,5066.5
...,...,...,...,...,...,...,...,...,...,...,...
5506,13/10/2023 15:31:39.241,5109.0,5110.5,5107.0,5110.5,5081.39,5072.26,5097.64,5081.39,-370,5107.0
5507,13/10/2023 15:37:05.486,5113.5,5116.0,5112.0,5112.0,5082.46,5072.26,5097.75,5082.46,80,5116.0
5508,13/10/2023 16:12:08.829,5106.0,5107.5,5103.0,5107.5,5084.52,5072.26,5097.93,5084.52,30,5103.0
5509,13/10/2023 16:44:31.211,5106.0,5108.0,5104.5,5104.5,5084.55,5072.26,5097.93,5084.55,-190,5108.0


<h1>Probabidade ao pivotiar para cima, virar um brik e romper o topo no 4r cheio</h1>

In [104]:
reversao = 0
gain = 0

for i in range(len(topos_fundos_agressao_4r_cheio_['Data'])-6):
    if (topos_fundos_agressao_4r_cheio_['Topos/fundos 4r_cheio'][i+2] > topos_fundos_agressao_4r_cheio_['Topos/fundos 4r_cheio'][i]) and (topos_fundos_agressao_4r_cheio_['Topos/fundos 4r_cheio'][i+1] < topos_fundos_agressao_4r_cheio_['Topos/fundos 4r_cheio'][i-1]) and (topos_fundos_agressao_4r_cheio_['Topos/fundos 4r_cheio'][i+3] > topos_fundos_agressao_4r_cheio_['Topos/fundos 4r_cheio'][i+1]) and (topos_fundos_agressao_4r_cheio_['Topos/fundos 4r_cheio'][i+3] < topos_fundos_agressao_4r_cheio_['Topos/fundos 4r_cheio'][i+2])\
    and topos_fundos_agressao_4r_cheio_['Quantidade de Agressão - 4R-cheio'][i+2] < 0:
        reversao +=1
        if (topos_fundos_agressao_4r_cheio_['Quantidade de Agressão - 4R-cheio'][i+4] > topos_fundos_agressao_4r_cheio_['Quantidade de Agressão - 4R-cheio'][i+2]) or (topos_fundos_agressao_4r_cheio_['Quantidade de Agressão - 4R-cheio'][i+5] > topos_fundos_agressao_4r_cheio_['Quantidade de Agressão - 4R-cheio'][i+2] and topos_fundos_agressao_4r_cheio_['Quantidade de Agressão - 4R-cheio'][i+5] > topos_fundos_agressao_4r_cheio_['Quantidade de Agressão - 4R-cheio'][i+3]):
            gain+= 1
            
print(gain/reversao)

0.8090452261306532


In [102]:
reversao = 0
gain = 0
for i in range(len(topos_fundos_agressao_4r_cheio_['Data'])-6):
    if (topos_fundos_agressao_4r_cheio_['Topos/fundos 4r_cheio'][i+2] > topos_fundos_agressao_4r_cheio_['Topos/fundos 4r_cheio'][i]) and \
    (topos_fundos_agressao_4r_cheio_['Fechamento'][i+3] - topos_fundos_agressao_4r_cheio_['Abertura'][i+3]) >0  and topos_fundos_agressao_4r_cheio_['Quantidade de Agressão - 4R-cheio'][i+3] <0:
        reversao += 1
        if topos_fundos_agressao_4r_cheio_['Topos/fundos 4r_cheio'][i+4] - topos_fundos_agressao_4r_cheio_['Fechamento'][i+3] >5:
            gain +=1
print(gain/reversao)            

0.3361581920903955


In [ ]:
reversao = 0
gain = 0
for i in range(len(topos_fundos_agressao_4r_cheio_['Data'])-6):
    if 

<h1>Estudo dolar 48r</h1>

In [ ]:
brick_48r = pd.read_excel('brick_48r.xlsx')
brick_48r = brick_48r[::-1]
brick_48r = brick_48r.reset_index()
del brick_48r['index']
condicao_brick = brick_48r['Abertura'] <=  brick_48r['Fechamento']
brick_48r.loc[condicao_brick,'Dia'] = 1
brick_48r.loc[~condicao_brick,'Dia'] = 0
brick_48r

,Data,Abertura,Máxima,Mínima,Fechamento,VWAP D,Prior Cote,VWAP S,VWAP D.1,Dia
0,28/04/2021 09:00:48.255,5452.0,5458.0,5428.5,5428.5,5439.36,5462.88,5439.36,5439.36,0.0
1,28/04/2021 09:34:53.378,5428.5,5430.0,5405.0,5405.0,5428.73,5462.88,5428.73,5428.73,0.0
2,28/04/2021 10:06:08.033,5405.0,5421.5,5381.5,5381.5,5408.98,5462.88,5408.98,5408.98,0.0
3,28/04/2021 15:21:48.186,5381.5,5396.0,5358.0,5358.0,5405.30,5462.88,5405.30,5405.30,0.0
4,28/04/2021 15:58:36.154,5358.0,5380.0,5334.5,5334.5,5384.06,5462.88,5384.06,5384.06,0.0
...,...,...,...,...,...,...,...,...,...,...
1459,09/10/2023 14:54:49.093,5146.5,5157.5,5123.0,5123.0,5142.17,5175.56,5142.17,5142.17,0.0
1460,10/10/2023 09:43:05.614,5123.0,5126.5,5099.5,5099.5,5115.32,5146.82,5129.85,5115.32,0.0
1461,10/10/2023 11:05:54.647,5099.5,5102.0,5076.0,5076.0,5096.83,5146.82,5109.38,5096.83,0.0
1462,10/10/2023 16:36:23.203,5076.0,5077.0,5052.5,5052.5,5093.84,5146.82,5106.15,5093.84,0.0


<h1>Probabidade de ser igual ao anterior 48R</h1>

In [ ]:
igual_48 = 0
for i in range(1463):
    d_1 = brick_48r['Dia'][i]
    d_2 = brick_48r['Dia'][i+1]
    if d_1 == d_2:      
        igual_48 += 1 

print(igual_48/1463)

0.6739576213260424


<h1>Probabidade de virar um brick e o próximo ser igual ao anterior 48R</h1>

In [ ]:
igual = 0
total = 0
_3_bricks = 0
for i in range(1461):
    d1 = brick_48r['Dia'][i]
    d2 = brick_48r['Dia'][i+1]
    d3 = brick_48r['Dia'][i+2]
    d4 = brick_48r['Dia'][i+3]
    if d1 != d2:        
        total += 1
        if d2 == d3:
            igual+= 1 
            if d3 == d4:
                _3_bricks += 1
print(igual/total,_3_bricks/total)

0.657563025210084 0.45588235294117646


In [1]:
print('##### RESPONDA APENAS SIM OU NÃO PARA AS PERGUNTAS #####')
pergunta_1 = input("Você matou a vítima?")
pergunta_2 = input("Você esteve com ela no dia de sua morte?")
pergunta_3 = input("Você esteve no local do crime?")
pergunta_4 = input("Você mora perto da vítima?")
pergunta_5 = input("Você devia dinheiro para a vítima?")
sentenca = None

if pergunta_1.lower() == 'sim':
      sentenca = 'Assassina'
elif pergunta_2.lower() == 'sim' or pergunta_3.lower() == 'sim':
      sentenca = 'Suspeito'
elif pergunta_4.lower() == 'sim':
      sentenca = 'Cúmplice'
else:
      sentenca = 'Inocente'

print(f'A pessoa é {sentenca}')

##### RESPONDA APENAS SIM OU NÃO PARA AS PERGUNTAS #####


Você matou a vítima? s
Você esteve com ela no dia de sua morte? s
Você esteve no local do crime? sim
Você mora perto da vítima? Sim
Você devia dinheiro para a vítima? sim


A pessoa é Suspeito
